In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from scipy.optimize import minimize

def state(t0, p0, l0, t1, p1, l1):
    """ función para crear el estado U(t0,p0,l0).kron(U(t1,p1,l1)).|00> + |11>)/(sqrt(2)) """
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0,1)
    # qc.x(0)
    qc.u(t0, p0, l0, 0)
    qc.u(t1, p1, l1, 1)
    return Statevector.from_instruction(qc)


def state_cepa(x0, y0, z0, x1, y1, z1):
    """Función para  crear el estado U(x0,y0,z0).kron(U(x1,y1,z1)).|00>"""
    qc = QuantumCircuit(2)
    qc.u(x0, y0, z0, 0)
    qc.u(x1, y1, z1, 1)
    return Statevector.from_instruction(qc)


def d_fs(x, y):
    """distancia flubini study """
    return np.arccos(np.real(x.inner(y)))

def cost(params):
    """Función de costo: 
    Muestra los parametro para cuando la distacia de flubini study es minima
     ¨"""
    
    t0, p0, l0, t1, p1, l1, x0, y0, z0, x1, y1, z1 = params
    entangled_state = state(t0, p0, l0, t1, p1, l1)
    separable_state = state_cepa(x0, y0, z0, x1, y1, z1)
    return d_fs(entangled_state, separable_state)

# Rango de parámetros
bounds = [
    (0, np.pi),      # t0
    (0, 2*np.pi),    # p0
    (0, 2*np.pi),    # l0
    (0, np.pi),      # t1
    (0, 2*np.pi),    # p1
    (0, 2*np.pi),    # l1
    (0, np.pi),      # x0
    (0, 2*np.pi),    # y0
    (0, 2*np.pi),    # z0
    (0, np.pi),      # x1
    (0, 2*np.pi),    # y1
    (0, 2*np.pi),    # z1
]

# Valores iniciales aleatorios
x0 = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds])

# Minimización
res = minimize(cost, x0, bounds=bounds, method='L-BFGS-B')

print("Distancia mínima encontrada (radianes):", res.fun)
print("Parámetros óptimos:", res.x)

# Si quieres convertir la distancia mínima a grados:
print("Distancia mínima (grados):", np.degrees(res.fun))


Distancia mínima encontrada (radianes): 0.7853981634994297
Parámetros óptimos: [2.33432095 0.54752847 4.66523971 1.59975489 4.49808118 6.27243852
 1.69364684 6.28318531 0.54277955 2.17066006 5.09896008 4.11447247]
Distancia mínima (grados): 45.000000005843106


In [2]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from scipy.optimize import differential_evolution

"""
La mismas funciones pero con otro método para minimizar. 
"""
# Estado entrelazado con compuertas H y CX + U en cada qubit
def state(t0, p0, l0, t1, p1, l1):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    qc.u(t0, p0, l0, 0)
    qc.u(t1, p1, l1, 1)
    return Statevector.from_instruction(qc)

# Estado separable con compuertas U en cada qubit
def state_cepa(x0, y0, z0, x1, y1, z1):
    qc = QuantumCircuit(2)
    qc.u(x0, y0, z0, 0)
    qc.u(x1, y1, z1, 1)
    return Statevector.from_instruction(qc)

# Distancia de Fubini-Study
def d_fs(x, y):
    return np.arccos(np.real(x.inner(y)))

# Función de costo para minimizar
def cost(params):
    t0, p0, l0, t1, p1, l1, x0, y0, z0, x1, y1, z1 = params
    entangled_state = state(t0, p0, l0, t1, p1, l1)
    separable_state = state_cepa(x0, y0, z0, x1, y1, z1)
    return d_fs(entangled_state, separable_state)

# Definición de los límites de los parámetros
bounds = [
    (0, np.pi),      # t0
    (0, 2*np.pi),    # p0
    (0, 2*np.pi),    # l0
    (0, np.pi),      # t1
    (0, 2*np.pi),    # p1
    (0, 2*np.pi),    # l1
    (0, np.pi),      # x0
    (0, 2*np.pi),    # y0
    (0, 2*np.pi),    # z0
    (0, np.pi),      # x1
    (0, 2*np.pi),    # y1
    (0, 2*np.pi),    # z1
]

# Ejecutar DE
result = differential_evolution(cost, bounds, strategy='best1bin', maxiter=300, popsize=20, tol=1e-6, disp=True)

# Mostrar resultados
print("\n📌 Distancia mínima encontrada (radianes):", result.fun)
print("📌 Distancia mínima (grados):", np.degrees(result.fun))
print("📌 Parámetros óptimos:")
for name, value in zip(['t0', 'p0', 'l0', 't1', 'p1', 'l1', 'x0', 'y0', 'z0', 'x1', 'y1', 'z1'], result.x):
    print(f"{name} = {value:.4f}")


differential_evolution step 1: f(x)= 0.7871664224616589
differential_evolution step 2: f(x)= 0.7871664224616589
differential_evolution step 3: f(x)= 0.785781421057916
differential_evolution step 4: f(x)= 0.785781421057916
differential_evolution step 5: f(x)= 0.785781421057916
differential_evolution step 6: f(x)= 0.785781421057916
differential_evolution step 7: f(x)= 0.785781421057916
differential_evolution step 8: f(x)= 0.785781421057916
differential_evolution step 9: f(x)= 0.785781421057916
differential_evolution step 10: f(x)= 0.785781421057916
differential_evolution step 11: f(x)= 0.785781421057916
differential_evolution step 12: f(x)= 0.785781421057916
differential_evolution step 13: f(x)= 0.785781421057916
differential_evolution step 14: f(x)= 0.785781421057916
differential_evolution step 15: f(x)= 0.785781421057916
differential_evolution step 16: f(x)= 0.785781421057916
differential_evolution step 17: f(x)= 0.785781421057916
differential_evolution step 18: f(x)= 0.785781421057916

In [3]:
def state(t0, p0, l0, t1, p1, l1):
    """
    Crear el estado sx.ecr.U.U'|00>, este estado es el maximamente entrelazado con puertas nativas. 
    """
    qc = QuantumCircuit(2)
    qc.sx(0)
    qc.ecr(0, 1)
    qc.u(t0, p0, l0, 0)
    qc.u(t1, p1, l1, 1)
    return Statevector.from_instruction(qc)

# Estado separable con compuertas U en cada qubit
def state_cepa(x0, y0, z0, x1, y1, z1):
    qc = QuantumCircuit(2)
    qc.u(x0, y0, z0, 0)
    qc.u(x1, y1, z1, 1)
    return Statevector.from_instruction(qc)

In [4]:
x0 = np.random.uniform([b[0] for b in bounds], [b[1] for b in bounds])

# Minimización
res = minimize(cost, x0, bounds=bounds, method='L-BFGS-B')

print("Distancia mínima encontrada (radianes):", res.fun)
print("Parámetros óptimos:", res.x)

# Si quieres convertir la distancia mínima a grados:
print("Distancia mínima (grados):", np.degrees(res.fun))


Distancia mínima encontrada (radianes): 0.7853981634515356
Parámetros óptimos: [0.80251976 5.75555806 5.35624177 1.80220232 0.37697154 0.73371622
 2.67432058 4.39600111 0.33896147 3.00727076 1.34106664 1.63840691]
Distancia mínima (grados): 45.000000003098975


In [5]:
(t0, p0, l0, t1, p1, l1)= res.x[6:12]
(x0, y0, z0, x1, y1, z1)= res.x[0:6]
params_00  = res.x[6:12]
theta = res.x
def gamma(k):
    """Función para encontarar el camino de estados que se encuentran en la geodesica"""
    theta =  res.fun
    vector = np.cos(k)*state_cepa(x0, y0, z0, x1, y1, z1) + np.sin(k)*(state(t0, p0, l0, t1, p1, l1) - np.cos(theta)*state_cepa(x0, y0, z0, x1, y1, z1) )/(np.sin(theta))
    
    return vector.data

In [6]:
from qiskit.quantum_info import Operator,Statevector
############################################################
# Distancia de Fubini-Study
def d_fs(x, y):
    return np.arccos(np.real(x.inner(y)))

#############################################################
#  Operators

II = Operator.from_label("II")
XI = Operator.from_label("XI")
ZI = Operator.from_label("ZI")
SXI= Operator.from_label("XI").power(0.5)

IX = Operator.from_label("IX")
IZ = Operator.from_label("IZ")
ISX= Operator.from_label("IX").power(0.5)

XX = Operator.from_label("XX")
ZZ = Operator.from_label("ZZ")
ISX_ISX = (Operator.from_label("X").power(0.5)).tensor(Operator.from_label("X").power(0.5)) 

###################################################################
# Vectors 

initial = Statevector.from_label("00")

cir = QuantumCircuit(2)
cir.sx(0)
cir.ecr(0,1)

final = Statevector(cir)


###################################################################
gates_nativas = [II,XI,ZI,SXI,IX,IZ,ISX, XX, ZZ, ISX_ISX]
gates_str =     ["II","XI","ZI","SXI","IX","IZ","ISX", "XX", "ZZ", "ISX_ISX"]

uno = []
for i in gates_nativas:
    U = np.matmul(i.data, initial.data)
    uno.append(U)

"""Distacia en grados para una compuerta actuando en cada qubit |00>"""
s = 0
for i in uno:
    s += 1
    dis = d_fs(Statevector(i),final)
    print(  np.degrees( dis), " ", gates_str[s-1]  )    
#############################################################################  
        

69.29518894536457   II
69.29518894536457   XI
69.29518894536457   ZI
90.0   SXI
69.29518894536457   IX
69.29518894536457   IZ
90.0   ISX
69.29518894536457   XX
69.29518894536457   ZZ
69.29518894536456   ISX_ISX


In [7]:
# diatancia flubini staudy y su compuertas para le estado |00> y el estado finala con la ecr


dos = []
for i in gates_nativas:
    for j in uno:
        UU = np.matmul(i.data,j)   
        dos.append(UU)
            
            
gate_dos = []
for i in gates_str:
    for j in gates_str:
        gate_dos.append([i,j])      
        
l = 0
for i in dos:
    l += 1
    dis = d_fs(Statevector(i),final)
    print(  np.degrees( dis), "   ",  gate_dos[l-1])                 
            

69.29518894536457     ['II', 'II']
69.29518894536457     ['II', 'XI']
69.29518894536457     ['II', 'ZI']
90.0     ['II', 'SXI']
69.29518894536457     ['II', 'IX']
69.29518894536457     ['II', 'IZ']
90.0     ['II', 'ISX']
69.29518894536457     ['II', 'XX']
69.29518894536457     ['II', 'ZZ']
69.29518894536456     ['II', 'ISX_ISX']
69.29518894536457     ['XI', 'II']
69.29518894536457     ['XI', 'XI']
69.29518894536457     ['XI', 'ZI']
45.0     ['XI', 'SXI']
69.29518894536457     ['XI', 'IX']
69.29518894536457     ['XI', 'IZ']
45.0     ['XI', 'ISX']
69.29518894536457     ['XI', 'XX']
69.29518894536457     ['XI', 'ZZ']
69.29518894536456     ['XI', 'ISX_ISX']
69.29518894536457     ['ZI', 'II']
110.70481105463544     ['ZI', 'XI']
69.29518894536457     ['ZI', 'ZI']
90.0     ['ZI', 'SXI']
69.29518894536457     ['ZI', 'IX']
69.29518894536457     ['ZI', 'IZ']
90.0     ['ZI', 'ISX']
110.70481105463544     ['ZI', 'XX']
69.29518894536457     ['ZI', 'ZZ']
110.70481105463544     ['ZI', 'ISX_ISX']
90.0

In [ ]:
tres = []
for i in gates_nativas:
    for j in dos:
        UUU = np.matmul(i.data,j)   
        tres.append(UUU)

            
gate_tres = []
for i in gates_str:
    for j in gates_str:
        for k in gates_str:
            gate_tres.append([i,j,k])      
        
l = 0
for i in tres:
    l += 1
    dis = d_fs(Statevector(i),final)
    print(  np.degrees( dis), "   ",  gate_tres[l-1])    

69.29518894536457     ['II', 'II', 'II']
69.29518894536457     ['II', 'II', 'XI']
69.29518894536457     ['II', 'II', 'ZI']
90.0     ['II', 'II', 'SXI']
69.29518894536457     ['II', 'II', 'IX']
69.29518894536457     ['II', 'II', 'IZ']
90.0     ['II', 'II', 'ISX']
69.29518894536457     ['II', 'II', 'XX']
69.29518894536457     ['II', 'II', 'ZZ']
69.29518894536456     ['II', 'II', 'ISX_ISX']
69.29518894536457     ['II', 'XI', 'II']
69.29518894536457     ['II', 'XI', 'XI']
69.29518894536457     ['II', 'XI', 'ZI']
45.0     ['II', 'XI', 'SXI']
69.29518894536457     ['II', 'XI', 'IX']
69.29518894536457     ['II', 'XI', 'IZ']
45.0     ['II', 'XI', 'ISX']
69.29518894536457     ['II', 'XI', 'XX']
69.29518894536457     ['II', 'XI', 'ZZ']
69.29518894536456     ['II', 'XI', 'ISX_ISX']
69.29518894536457     ['II', 'ZI', 'II']
110.70481105463544     ['II', 'ZI', 'XI']
69.29518894536457     ['II', 'ZI', 'ZI']
90.0     ['II', 'ZI', 'SXI']
69.29518894536457     ['II', 'ZI', 'IX']
69.29518894536457     ['

In [9]:
# para el ghz

# theta(000,GHZ) = 0.7853
# theta(000, W) = 1.57079
# theta(GHZ,W) = 1.57079

GHZ = QuantumCircuit(3)
GHZ.h(0)
GHZ.cx(0,1)
GHZ.cx(1,2)

ghz = Statevector(GHZ)

initial_state = Statevector.from_label("000")

w = Statevector([0,1,1,0,1,0,0,0]/np.linalg.norm(np.array([0,1,1,0,1,0,0,0])))

dis_ghz = d_fs(ghz,initial_state)
dis_w   = d_fs(ghz,w)
print(f"la distancia en drados para el ghz es: {np.degrees(dis_ghz)},  para el estado w: {np.degrees(dis_w)}")

print(f"distancia entre los dos estados gaz y w :  {np.degrees(d_fs(w,ghz))}")

la distancia en drados para el ghz es: 45.00000000000001,  para el estado w: 90.0
distancia entre los dos estados gaz y w :  90.0
